# Lab Sheet 3: Extracting Word Frequency Vectors with Spark

These tasks are for working in the lab session and during the week. We will use the same data as last week (19 files in './City-Data-Science/library/') and use some more RDD functions. We will apply two different approaches to create and use fixed size vectors.

First update the repo.

In [4]:
!git clone https://github.com/tweyde/City-Data-Science.git
%cd City-Data-Science/
!git pull https://github.com/tweyde/City-Data-Science.git
%cd ..

fatal: destination path 'City-Data-Science' already exists and is not an empty directory.
/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work/City-Data-Science
From https://github.com/tweyde/City-Data-Science
 * branch            HEAD       -> FETCH_HEAD
Already up-to-date.
/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work


Here is code from last week that we run first, and then extend. 

In [106]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

testRDD = sc.parallelize(["hadoop is fast", "hive is sql on hdfs", "spark is superfast", "spark is awesome"])

testRDD.collect()

# ['hadoop is fast',
# 'hive is sql on hdfs',
# 'spark is superfast',
# 'spark is awesome']


#wc = testRDD.map(lambda line:line.split(" "));
wc = testRDD.map(lambda line:line + "hello");

wc.collect()

#[['hadoop', 'is', 'fast'],
# ['hive', 'is', 'sql', 'on', 'hdfs'],
# ['spark', 'is', 'superfast'],
# ['spark', 'is', 'awesome']]

# ['hadoop is fasthello',
#  'hive is sql on hdfshello',
#  'spark is superfasthello',
#  'spark is awesomehello']


#fm = testRDD.flatMap(lambda line:line.split(" "));
fm = testRDD.flatMap(lambda line:line + "hello");
fm.collect()

# Flatmap flattens each component of the inner list into separate components. If there is no inner list then each component of the list is flattened into its component parts i.e. each character)

# ['hadoop',
#  'is',
#  'fast',
#  'hive',
#  'is',
#  'sql',
#  'on',
#  'hdfs',
#  'spark',
#  'is',
#  'superfast',
#  'spark',
#  'is',
#  'awesome']


# ['h',
#  'a',
#  'd',
#  'o',
#  'o',
#  'p',
#  ' ',
#  'i',
#  's',
#  ' ',
#  'f',
#  'a',
#  's',
#  't',
#  'h',
#  'e',
#  'l',
#  'l',
#  'o',
#  'h',
#  'i',
#  'v',
#  'e',
#  ' ',
#  'i',
#  's',
#  ' ',
#  's',
#  'q',
#  'l',
#  ' ',
#  'o',
#  'n',
#  ' ',
#  'h',
#  'd',
#  'f',
#  's',
#  'h',
#  'e',
#  'l',
#  'l',
#  'o',
#  's',
#  'p',
#  'a',
#  'r',
#  'k',
#  ' ',
#  'i',
#  's',
#  ' ',
#  's',
#  'u',
#  'p',
#  'e',
#  'r',
#  'f',
#  'a',
#  's',
#  't',
#  'h',
#  'e',
#  'l',
#  'l',
#  'o',
#  's',
#  'p',
#  'a',
#  'r',
#  'k',
#  ' ',
#  'i',
#  's',
#  ' ',
#  'a',
#  'w',
#  'e',
#  's',
#  'o',
#  'm',
#  'e',
#  'h',
#  'e',
#  'l',
#  'l',
#  'o']

['h',
 'a',
 'd',
 'o',
 'o',
 'p',
 ' ',
 'i',
 's',
 ' ',
 'f',
 'a',
 's',
 't',
 'h',
 'e',
 'l',
 'l',
 'o',
 'h',
 'i',
 'v',
 'e',
 ' ',
 'i',
 's',
 ' ',
 's',
 'q',
 'l',
 ' ',
 'o',
 'n',
 ' ',
 'h',
 'd',
 'f',
 's',
 'h',
 'e',
 'l',
 'l',
 'o',
 's',
 'p',
 'a',
 'r',
 'k',
 ' ',
 'i',
 's',
 ' ',
 's',
 'u',
 'p',
 'e',
 'r',
 'f',
 'a',
 's',
 't',
 'h',
 'e',
 'l',
 'l',
 'o',
 's',
 'p',
 'a',
 'r',
 'k',
 ' ',
 'i',
 's',
 ' ',
 'a',
 'w',
 'e',
 's',
 'o',
 'm',
 'e',
 'h',
 'e',
 'l',
 'l',
 'o']

In [126]:
import re 

def stripFinalS( word ):
    word = word.lower() # lower case
    if len(word) >0 and word[-1] == 's': # check for final letter
        return word[:-1]
    else:
        return word
    
def splitFileWords(filenameContent): # your splitting function
    f,c = filenameContent # split the input tuple  
    fwLst = [] # the new list for (filename,word) tuples
    wLst = re.split('\W+',c) # <<< now create a word list wLst
    for w in wLst : # iterate through the list
        fwLst.append((f,stripFinalS(w))) # and append (f,w) to the 
    return fwLst #return a list of (f,w) tuples 

from pyspark import SparkContext

sc = SparkContext.getOrCreate()
!pwd
dirPath = './City-Data-Science/library/' #  path
ft_RDD = sc.wholeTextFiles(dirPath) # create an RDD with wholeTextFiles
#ft_RDD.take(1)
fnt_RDD = ft_RDD.map(lambda ft: (re.split('[/\.]',ft[0])[-2],ft[1])) # just take filenames, 
#                                                 # drop path and extension for readability

#fnt_RDD.take(10)

fw_RDD1 = fnt_RDD.flatMap(splitFileWords) # split words per file, strip final 's'
fw_RDD1.take(10)
# with flatmap fw_RDD1 looks like below with map each book is in a separate list so you have a list of lists structure
# [('prideandpredjudice', ''),
#  ('prideandpredjudice', 'the'),
#  ('prideandpredjudice', 'project'),
#  ('prideandpredjudice', 'gutenberg'),
#  ('prideandpredjudice', 'ebook'),
#  ('prideandpredjudice', 'of'),
#  ('prideandpredjudice', 'pride'),
#  ('prideandpredjudice', 'and'),
#  ('prideandpredjudice', 'prejudice'),
#  ('prideandpredjudice', 'by')]

fw_RDD = fw_RDD1.filter(lambda fw: fw[1] not in ['','project','gutenberg', 'ebook']) # get rid of some unwanted words
fw_RDD.take(10)
# output should look like this: [('emma', 'the'), ('emma', 'of'), ('emma', 'emma')]

/gpfs/fs01/user/sb1c-2a2fc727357cc3-3b10766cfe3e/notebook/work


[('prideandpredjudice', 'the'),
 ('prideandpredjudice', 'of'),
 ('prideandpredjudice', 'pride'),
 ('prideandpredjudice', 'and'),
 ('prideandpredjudice', 'prejudice'),
 ('prideandpredjudice', 'by'),
 ('prideandpredjudice', 'jane'),
 ('prideandpredjudice', 'austen'),
 ('prideandpredjudice', 'thi'),
 ('prideandpredjudice', 'i')]

## 1) Warm-up
Let's start with a few small tasks, to become more fluent with RDDs and lambda expressions.

a) Count the number of documents.
b) Determine the number distinct words in total (the vocabulary size) using RDD.distinct(). This involves removing the fs from the (f,w) pairs and geting getting the RDD size (with RDD.count()). 
c) Get the number of words (including repeated ones) per book. 
d) Determine the number of distinct words per book. This involves determining the disting (f,w) pairs, geting a list of words per file, and getting the list size.
e) Count the average number of occurences per word per file (words/vocabulary). Use RDD.join() to get both numbers into one RDD. 

In [108]:
from operator import add
# a) Library size
print("Number of documents: ",ft_RDD.count()) # count the number of docs

# b) Vocabulary size
w_RDD = fw_RDD.map( lambda ft: ft[1] ) # remove the file names, keep just the words
w_RDDu = w_RDD.distinct() # keep only one unique instance of every word
print('Total vocabulary size: ',w_RDDu.count()) # 
print('Total vocabulary size (v2): ',fw_RDD.values().distinct().count())


# c) words per book
f1_RDD = fw_RDD.map(lambda fw: (fw[0],1)) # swap and wrap (f,w) to (f,1)
f1_RDD.take(3)
fc_RDD = f1_RDD.reduceByKey(add) # add the 1s up
print('Words per book: ',fc_RDD.take(3))
word_doc_count = fw_RDD.countByKey()
print(word_doc_count)
# or alternatively (both versions should produce the same output):
print('Words per book (v2): ',fw_RDD.countByKey())


from operator import add
fw_RDDu = fw_RDD.distinct() # get unique (f,w) pairs - i.e. evey word only once per file. I use postfix u to mark 'unique'
f1_RDDu = fw_RDDu.map(lambda fw: (fw[0],1)) # wrap (f,w) to (f,1)
fcu_RDD = f1_RDDu.reduceByKey(add) # add the 1s up
print('Vocabulary per book: ',fcu_RDD.take(3)) 
# or, again, shorter:
print('Vocabulary per book (v2): ',fw_RDD.distinct().countByKey())


Number of documents:  18
Total vocabulary size:  23368
Total vocabulary size (v2):  23368
Words per book:  [('othello', 30617), ('macbeth', 20810), ('lady_susan', 26109)]
defaultdict(<class 'int'>, {'macbeth': 20810, 'othello': 30617, 'prideandpredjudice': 125212, 'julius_cesar': 22959, 'lady_susan': 26109, 'midsummer': 19814, 'senseandsensibility': 123066, 'mansfield_park': 163509, 'merchant_of_venice': 24772, 'richard_III': 33785, 'king_lear': 30119, 'tempest': 27891, 'persuasion': 83680, 'northanger_abbey': 77582, 'hamlet': 34518, 'romeo_and_juliet': 29538, 'henry_V': 29900, 'emma': 163979})
Words per book (v2):  defaultdict(<class 'int'>, {'macbeth': 20810, 'othello': 30617, 'prideandpredjudice': 125212, 'julius_cesar': 22959, 'lady_susan': 26109, 'midsummer': 19814, 'senseandsensibility': 123066, 'mansfield_park': 163509, 'merchant_of_venice': 24772, 'richard_III': 33785, 'king_lear': 30119, 'tempest': 27891, 'persuasion': 83680, 'northanger_abbey': 77582, 'hamlet': 34518, 'romeo_

In [109]:
from operator import add
# a) Library size
print("Number of documents: ",ft_RDD.count()) # count the number of docs

# b) Vocabulary size
w_RDD = fw_RDD.map( lambda ft: ft[1] ) # remove the file names, keep just the words
w_RDDu = w_RDD.distinct() # keep only one unique instance of every word
print('Total vocabulary size: ',w_RDDu.count()) # 

# c) words per book
f1_RDD = fw_RDD.map(lambda fw: (fw[0],1)) # swap and wrap (f,w) to (f,1)
fc_RDD = f1_RDD.reduceByKey(add) # add the 1s up
print('Words per book: ',fc_RDD.take(3))

# extra task: try also to express this with one function that appeared in the lecture today
print('Words per book: ',fc_RDD.take(3))
word_doc_count = fw_RDD.countByKey()
print(word_doc_count)


# d) Vocabulary per book
from operator import add
fw_RDDu = fw_RDD.distinct() # get unique (f,w) pairs - i.e. evey word only once per file. I use postfix u to mark 'unique'
f1_RDDu = fw_RDDu.map(lambda fw: (fw[0],1)) # wrap (f,w) to (f,1)
fcu_RDD = f1_RDDu.reduceByKey(add) # add the 1s up
print('Vocabulary per book: ',fcu_RDD.take(3)) 
# extra task: try also replacing the map and reduce by one function 
# or, again, shorter:
print('Vocabulary per book (v2): ',fw_RDD.distinct().countByKey())

fc_RDD.take(3)
fcu_RDD.take(3)


# e) Average occurences of words per book (i.e. words/vocab per book)
f_wv_RDD = fc_RDD.join(fcu_RDD) # join the two RDDs to get (f,(w,v)) tuples

# fc_RDD looks like this 
# [('macbeth', 20810), ('othello', 30617), ('lady_susan', 26109)]
# fcu_RDD looks like this
# [('othello', 4077), ('macbeth', 3654), ('lady_susan', 3097)]
print(f_wv_RDD.take(3)) 
f_awo_RDD = f_wv_RDD.map(lambda f_wv: (f_wv[0], f_wv[1][0]/f_wv[1][1] )) # this is the tricky part. 
            # Resolve nested tuples in the lambda to get (filename,words/vocab) tuples
print('Average word occurences: ',f_awo_RDD.take(3))
# should look like this [('henry_V', 6.237027812370278), ('king_lear', 7.815661103979461), ('lady_susan', 8.531763947113834)]


Number of documents:  18
Total vocabulary size:  23368
Words per book:  [('othello', 30617), ('macbeth', 20810), ('lady_susan', 26109)]
Words per book:  [('macbeth', 20810), ('othello', 30617), ('lady_susan', 26109)]
defaultdict(<class 'int'>, {'macbeth': 20810, 'othello': 30617, 'prideandpredjudice': 125212, 'julius_cesar': 22959, 'lady_susan': 26109, 'midsummer': 19814, 'senseandsensibility': 123066, 'mansfield_park': 163509, 'merchant_of_venice': 24772, 'richard_III': 33785, 'king_lear': 30119, 'tempest': 27891, 'persuasion': 83680, 'northanger_abbey': 77582, 'hamlet': 34518, 'romeo_and_juliet': 29538, 'henry_V': 29900, 'emma': 163979})
Vocabulary per book:  [('macbeth', 3654), ('othello', 4077), ('lady_susan', 3097)]
Vocabulary per book (v2):  defaultdict(<class 'int'>, {'othello': 4077, 'mansfield_park': 7492, 'hamlet': 4447, 'prideandpredjudice': 6181, 'julius_cesar': 2834, 'lady_susan': 3097, 'king_lear': 3892, 'senseandsensibility': 6073, 'merchant_of_venice': 3686, 'richard_II


## 2) Fixed vectors: Reduced vocabulary approach

The first task in this lab is to use a reduced vocabulary, only the stopwords from a list, to make sure that we have a fixed size vector. This is a common approach in stylometry. The problem is that some stopwords might not appear in some documents. We will deal with that by creating an RDD with ((f,w),0) tuples that we then merge with the ((f,w),count) RDD. 

Start by running the code above, then you can add 1s and use reduceByKey(add) like last week to get the counts of the words per filename.

Then, please make sure that all stopwords are present by creating a new RDD that contains the keys of the fw_RDD, i.e. the filenames, using the keys() method of class RDD. Then you can use flatMap to create a [((filname,stopword),0), ...] list, using a list comprehension. The 0s should not be 1s, as we don't want add to add extra counts.
The RDD with ((filename,stopword),0) tuples can then be merged with fw_RDD2 using union(). Then you can count as normal.

In [110]:
from operator import add

stopwlst = ['the','a','in','of','on','at','for','by','i','you','me'] # stopword list
fw_RDD2 = fw_RDD.filter(lambda x: x[1] in stopwlst) # filter, keeping only stopwords

# fw_RDD2 structure
# [('prideandpredjudice', 'the'),
#  ('prideandpredjudice', 'of'),
#  ('prideandpredjudice', 'by')]

fsw_0_RDD = fw_RDD.keys().flatMap(lambda f: [((f,sw),0) for sw in stopwlst])


# fsw_0_RDD structure
# [(('prideandpredjudice', 'the'), 0), (('prideandpredjudice', 'a'), 0), (('prideandpredjudice', 'in'), 0)]

######## CHECK LINE BELOW
fw_1_RDD = fw_RDD2.map(lambda fw: ((fw[0],fw[1]),1))  #<<< change (f,w) to ((f,w),1)


fw_10_RDD = fw_1_RDD.union(fsw_0_RDD) #<<< create the union on the two RDDs
print(fw_10_RDD.take(50))

[(('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'by'), 1), (('prideandpredjudice', 'i'), 1), (('prideandpredjudice', 'for'), 1), (('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'at'), 1), (('prideandpredjudice', 'you'), 1), (('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'at'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'by'), 1), (('prideandpredjudice', 'by'), 1), (('prideandpredjudice', 'i'), 1), (('prideandpredjudice', 'a'), 1), (('prideandpredjudice', 'a'), 1), (('prideandpredjudice', 'in'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'a'), 1), (('prideandpredjudice', 'in'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'a'), 1), (('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'a'), 1), (('prideandpredjudi

In [111]:
from operator import add

stopwlst = ['the','a','in','of','on','at','for','by','i','you','me'] # stopword list
fw_RDD2 = fw_RDD.filter(lambda x: x[1] in stopwlst) # filter, keeping only stopwords

# fw_RDD2 structure
# [('prideandpredjudice', 'the'),
#  ('prideandpredjudice', 'of'),
#  ('prideandpredjudice', 'by')]

fsw_0_RDD = fw_RDD.keys().flatMap(lambda f: [((f,sw),0) for sw in stopwlst])
print(fsw_0_RDD.take(3))

# fsw_0_RDD structure
# [(('prideandpredjudice', 'the'), 0), (('prideandpredjudice', 'a'), 0), (('prideandpredjudice', 'in'), 0)]

######## CHECK LINE BELOW
fw_1_RDD = fw_RDD2.map(lambda fw: ((fw[0],fw[1]),1))  #<<< change (f,w) to ((f,w),1)
print(fw_1_RDD.take(3))
fw_1_RDD = fw_RDD2.map(lambda fw: (fw,1))  #<<< change (f,w) to ((f,w),1)
print(fw_1_RDD.take(10))
# # fw_1_RDD structure
# # [(('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'by'), 1)]

fw_10_RDD = fw_1_RDD.union(fsw_0_RDD) #<<< create the union on the two RDDs
print(fw_10_RDD.take(3))

fw_c_RDD = fw_10_RDD.reduceByKey(add) #<<< count the words
print(fw_c_RDD.take(3))
# # output should look like this:
# #[(('emma', 'the'), 0), (('emma', 'a'), 0), (('emma', 'in'), 0)]
# #[(('emma', 'the'), 1), (('emma', 'of'), 1), (('emma', 'by'), 1)]
# #[(('emma', 'the'), 1), (('emma', 'of'), 1), (('emma', 'by'), 1)]
# #[(('emma', 'the'), 5380), (('emma', 'by'), 591), (('emma', 'you'), 2068)]

[(('prideandpredjudice', 'the'), 0), (('prideandpredjudice', 'a'), 0), (('prideandpredjudice', 'in'), 0)]
[(('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'by'), 1)]
[(('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'by'), 1), (('prideandpredjudice', 'i'), 1), (('prideandpredjudice', 'for'), 1), (('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'at'), 1), (('prideandpredjudice', 'you'), 1), (('prideandpredjudice', 'the'), 1)]
[(('prideandpredjudice', 'the'), 1), (('prideandpredjudice', 'of'), 1), (('prideandpredjudice', 'by'), 1)]
[(('mansfield_park', 'the'), 6382), (('midsummer', 'in'), 265), (('prideandpredjudice', 'in'), 1937)]


## 3) Creating sorted lists

As a next step, map the `((filename,word),count)` to `( filename, [ (word, count) ])` using the function `reGrpLst` to regroup and create a list. 

Then sort the [(word,count),...] lists in the values (i.e. 2nd part of the tuple) with the the words as keys. Have a [look at the Python docs](https://docs.python.org/3.5/library/functions.html?highlight=sorted#sorted) for how to do this. Hint: use a lambda that extracts the words as the key, e.g. `sorted(f_wdL[1], key = lambda wc: ... )`.   

In [112]:
def reGrpLst(fw_c): # we get a nested tuple
    fw,c = fw_c # split the outer tuple
    f,w = fw     # split the inner tuple
    return (f,[(w,c)]) # return (f,[(w,c)]) structure. Can be used verbatim, if your variable names match.


f_wcL_RDD = fw_c_RDD.map(reGrpLst) # apply reGrpLst
f_wcL2_RDD = f_wcL_RDD.reduceByKey(add) # create [(w,c), ... ,(w,c)] lists per file 
f_wcL2_RDD.take(5)
f_wcLsort_RDD = f_wcL2_RDD.map(lambda f_wcL: (f_wcL[0], sorted(f_wcL[1],key=lambda x: x[0]))) #<<< sort the word count lists by word
print(f_wcLsort_RDD.take(3)) 
f_wVec_RDD = f_wcLsort_RDD.map(lambda f_wc: (f_wc[0],[float(c) for w,c in f_wc[1]])) # remove the words from the wc pairs and convert the numbers to floats
f_wVec_RDD.take(3)
# output:
# [('lady_susan', [('a', 611), ('at', 161), ('by', 152), ('for', 262), ('i', 1106), ('in', 402), ('me', 200), ('of', 787), ...

[('othello', [('a', 668), ('at', 78), ('by', 131), ('for', 270), ('i', 1222), ('in', 349), ('me', 281), ('of', 542), ('on', 132), ('the', 820), ('you', 552)]), ('lady_susan', [('a', 611), ('at', 161), ('by', 152), ('for', 262), ('i', 1106), ('in', 402), ('me', 200), ('of', 787), ('on', 140), ('the', 784), ('you', 353)]), ('persuasion', [('a', 2404), ('at', 533), ('by', 418), ('for', 708), ('i', 1522), ('in', 1389), ('me', 188), ('of', 2570), ('on', 396), ('the', 3329), ('you', 628)])]


[('othello',
  [668.0,
   78.0,
   131.0,
   270.0,
   1222.0,
   349.0,
   281.0,
   542.0,
   132.0,
   820.0,
   552.0]),
 ('lady_susan',
  [611.0,
   161.0,
   152.0,
   262.0,
   1106.0,
   402.0,
   200.0,
   787.0,
   140.0,
   784.0,
   353.0]),
 ('persuasion',
  [2404.0,
   533.0,
   418.0,
   708.0,
   1522.0,
   1389.0,
   188.0,
   2570.0,
   396.0,
   3329.0,
   628.0])]

## 4) Clustering

Now we have feature vectors of fixed size, we can use KMeans as provided by Spark.

The files in our library are by two authors. After clustering, check if the cluters reflect authorship:

WILLIAM SHAKESPEARE: 
merchant_of_venice, 
richard_III, 
midsummer,
tempest,
romeoandjuliet,
othello,
henry_V,
macbeth,
king_lear,
julius_cesar,
hamlet

JANE AUSTEN
mansfield_park,
emma,
northanger_abbey,
lady_susan,
persuasion,
prideandpredjudice,
senseandsensibility

In [113]:

from math import sqrt

from pyspark.mllib.clustering import KMeans #, KMeansModel

print('f_wVec_RDD.take(2): ', f_wVec_RDD.take(1))
wVec_RDD = f_wVec_RDD.map(lambda f_wcl: f_wcl[1]) # strip the filenames, keep only the vectors
print(wVec_RDD.take(3))

# Build the model (cluster the data)
clusterModel = KMeans.train(wVec_RDD, 2, maxIterations=10, initializationMode="random")
print (clusterModel.centers)

# Assign the filenames to the clusters
fc_RDD = f_wVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
for s in fc_RDD.collect():
    print(s)

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusterModel.centers[clusterModel.predict(point)] # coordinate of center of each of the 2 clusters
    return sqrt(sum([x**2 for x in (point - center)])) # returns sum of the squares between point and its cluster center

WSSSE = wVec_RDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))
# now check if the clusters match the authors
# output:
#('lady_susan', 1)
#('macbeth', 1)
#('merchant_of_venice', 1)
#('othello', 1)
#('persuasion', 0)
#('emma', 0)


testRDD = sc.parallelize([1, 2, 4, 10])
testRDD1 = testRDD.reduce(lambda x,y: x + y)
print(testRDD1)

f_wVec_RDD.take(2):  [('othello', [668.0, 78.0, 131.0, 270.0, 1222.0, 349.0, 281.0, 542.0, 132.0, 820.0, 552.0])]
[[668.0, 78.0, 131.0, 270.0, 1222.0, 349.0, 281.0, 542.0, 132.0, 820.0, 552.0], [611.0, 161.0, 152.0, 262.0, 1106.0, 402.0, 200.0, 787.0, 140.0, 784.0, 353.0], [2404.0, 533.0, 418.0, 708.0, 1522.0, 1389.0, 188.0, 2570.0, 396.0, 3329.0, 628.0]]
[array([ 3452.        ,   802.        ,   604.5       ,  1125.5       ,
        2816.        ,  1906.33333333,   379.        ,  3609.33333333,
         616.16666667,  4507.5       ,  1321.83333333]), array([ 591.91666667,   86.75      ,  134.91666667,  236.83333333,
        929.25      ,  342.58333333,  217.08333333,  578.        ,
        108.08333333,  895.91666667,  423.08333333])]
('othello', 1)
('lady_susan', 1)
('persuasion', 0)
('macbeth', 1)
('merchant_of_venice', 1)
('mansfield_park', 0)
('julius_cesar', 1)
('emma', 0)
('tempest', 1)
('richard_III', 1)
('midsummer', 1)
('northanger_abbey', 0)
('senseandsensibility', 0)
('prid

## 4) Alternative approach: feature hashing

Instead of the previous appraoch, we now use feature hashing, as done last week.

In [114]:
def hashing_vectorizer(word_count_list, N):
    v = [0] * N  # create fixed size vector of 0s
    for word_count in word_count_list: 
        word, count =  word_count    # unpack tuple
        h = hash(word)              # get hash value
        v[h % N] = v[h % N] + count # add count
    return v # return hashed word vector

from operator import add

N = 10

# we use fw_RDD from the beginning with all the words, not just stopwords
fw_1_RDD = fw_RDD.map(lambda fw: (fw, 1))  #<<< change (f,w) to ((f,w),1)
#fw_1_RDD.take(3)                
fw_c_RDD = fw_1_RDD.reduceByKey(add) #as above
f_wcL_RDD = fw_c_RDD.map(reGrpLst) #as above
f_wcL_RDD.take(10)

f_wcL2_RDD = f_wcL_RDD.reduceByKey(add) #create [(w,c), ... ,(w,c)] lists per file 
f_wcL2_RDD.take(10)
f_wVec_RDD = f_wcL2_RDD.map(lambda f_wcl: (f_wcl[0], hashing_vectorizer(f_wcl[1], N))) # apply the hashing_vectorizer to the word-count list
print(f_wVec_RDD.take(3))
# output:
# [('henry_V', [2277, 2293, 1182, 1792, 2058, 1550, 787, 1821, 814, 1916, 902, 752, 1249, 1022, 888, 1702, 1357, 2886, 1007, 1645]), ('king_lear', [2149, 2217, 1010, 2167, 2331, 1372, 726, 1682, 747, 1623, 1470, 889, 1248, 1371, 1062, 1472, 1510, 2456, 1364, 1253]), ('lady_susan', [2015, 1850, 823, 1828, 2099, 1658, 704, 1656, 588, 1319, 1433, 789, 1051, 909, 748, 1236, 1290, 2195, 570, 1348])]

[('othello', [3210, 3107, 2080, 3437, 3237, 3216, 2510, 4440, 1962, 3418]), ('macbeth', [1964, 2024, 1877, 2148, 2010, 2234, 1610, 3167, 1339, 2437]), ('lady_susan', [3448, 2639, 1874, 2737, 2847, 2894, 1994, 3851, 1158, 2667])]


In [115]:
from math import sqrt

from pyspark.mllib.clustering import KMeans #, KMeansModel

print('f_wVec_RDD.take(2): ', f_wVec_RDD.take(1))
wVec_RDD = f_wVec_RDD.map(lambda f_wcl: f_wcl[1]) # strip the filenames
print(wVec_RDD.collect())

# Build the model (cluster the data)
clusterModel = KMeans.train(wVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the files to the clusters
fc_RDD = f_wVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1]))) 
for s in fc_RDD.collect():
    print(s)
    
# resusing 'error' function from abov
WSSSE = wVec_RDD.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

f_wVec_RDD.take(2):  [('macbeth', [1964, 2024, 1877, 2148, 2010, 2234, 1610, 3167, 1339, 2437])]
[[3210, 3107, 2080, 3437, 3237, 3216, 2510, 4440, 1962, 3418], [1964, 2024, 1877, 2148, 2010, 2234, 1610, 3167, 1339, 2437], [3448, 2639, 1874, 2737, 2847, 2894, 1994, 3851, 1158, 2667], [3619, 3106, 2258, 3538, 3393, 2844, 2236, 4138, 2111, 2876], [17073, 12753, 7663, 11839, 10487, 13163, 9662, 20753, 5831, 13842], [2307, 2412, 1824, 2844, 2726, 2764, 1921, 3863, 1401, 2710], [2039, 2084, 1604, 2005, 1986, 2244, 1637, 2750, 1253, 2212], [17312, 14009, 7456, 11714, 11005, 13677, 9499, 21236, 5907, 13397], [11344, 8607, 5039, 7593, 6855, 9336, 6130, 14833, 4520, 9423], [10493, 8084, 5098, 7615, 6689, 8377, 5797, 12660, 3987, 8782], [3195, 3043, 2241, 3201, 3040, 3009, 2469, 3814, 2354, 3172], [3179, 3045, 2431, 2814, 2946, 3252, 2144, 4707, 1821, 3561], [22749, 17943, 9771, 16800, 14250, 16454, 12948, 27140, 7448, 18006], [4228, 2689, 2212, 3057, 2638, 2747, 2266, 3548, 1881, 2625], [3753, 3

## 5) Neutralising document length: Normalised vectors

'Lady Susan' ends up reliably in the wrong cluster. A possible explanation is that it is shorter than the other Austen works. Try normalising the word counts, i.e. by dividing by their sum. That takes away the effect of length. What is the effect on the clustering?
    
You can use a list comprehension for the normalisation.

In [118]:
from pyspark.mllib.clustering import KMeans #, KMeansModel
nwVec_RDD = wVec_RDD.map(lambda v: ([count/sum(v) for count in v])) # provide a list comprehension that 
                            #normalises the values by dividing by the sum over the list
# nfwVec_RDD = f_wVec_RDD.map(lambda fc: (fc[0], [count/sum(fc[1]) for count in fc[1]]))    
    
print("Normalised vectors: ",nwVec_RDD.take(3))
print("Normalised vectors: ",f_wVec_RDD.take(3))

# print("Normalised vectors: ",nfwVec_RDD.take(3)) 


# Build the model (cluster the data)
clusterModel = KMeans.train(nwVec_RDD, 2, maxIterations=10, initializationMode="random")

# Assign the files to the clusters
# fc_RDD = nfwVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
# fc_RDD.take(3)

fc_RDD = f_wVec_RDD.map(lambda fv: (fv[0],clusterModel.predict(fv[1])))
f_wVec_RDD.take(3)
for s in fc_RDD.collect():
    print(s)
# output
# Normalised vectors:  [[0.07615384615384616, 0.07668896321070234, 0.03953177257525083, 0.05993311036789298, 
# ...                       
# ('henry_V', 0)
# ('king_lear', 0)
#('lady_susan', 1)
# ..

Normalised vectors:  [[0.09437770302739068, 0.0972609322441134, 0.09019702066314272, 0.10321960595867372, 0.09658817876021143, 0.10735223450264296, 0.07736665064872657, 0.1521864488226814, 0.06434406535319558, 0.11710716001922153], [0.10484371427638241, 0.10147957017343306, 0.0679361139236372, 0.11225789594016396, 0.10572557729366039, 0.10503968383577751, 0.08198059901361988, 0.1450174739523794, 0.06408204592220008, 0.11163732566874612], [0.1320617411620514, 0.1010762572293079, 0.07177601593320311, 0.10482975219273048, 0.10904285878432725, 0.1108430043280095, 0.07637213221494504, 0.14749703167490139, 0.044352522118809606, 0.10214868436171436]]
Normalised vectors:  [('othello', [3210, 3107, 2080, 3437, 3237, 3216, 2510, 4440, 1962, 3418]), ('macbeth', [1964, 2024, 1877, 2148, 2010, 2234, 1610, 3167, 1339, 2437]), ('lady_susan', [3448, 2639, 1874, 2737, 2847, 2894, 1994, 3851, 1158, 2667])]
('othello', 0)
('macbeth', 0)
('lady_susan', 0)
('king_lear', 0)
('senseandsensibility', 0)
('merc

## 6) Building an index

Starting from the fw_RDD we now start building the index and calculating the IDF values. Since we have the TF values alread, we only need to keep the unique filenames per word using [RDD.distinct()](https://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.distinct).  
Then we create a list of filenames. The length of the list is the document frequency DF per word.
From the DF value we can calculate the IDF value as log(18/DF) 

In [125]:
from operator import add
from math import log
fw_RDD.take(3)
fwu_RDD = fw_RDD.distinct() # get unique file/word pairs

wfl_RDD = fwu_RDD.map(lambda fw: (fw[1],[fw[0]])) # create (w,[f]) tuples 
wfL_RDD = wfl_RDD.reduceByKey(add) # concatenate the lists with 'add'
print(wfL_RDD.take(3))

wdf_RDD = wfL_RDD.map(lambda wfl: (wfl[0],len(wfl[0]))) # get the DF replacing the file list with its lenght
print("DF: ",wdf_RDD.take(3))
widf_RDD = wdf_RDD.map(lambda wdf: (wdf[0],log(18/wdf[1]))) # get he IDF by replacing DF with log(18/DF)
print("IDF: ",widf_RDD.take(3))
# ouptut:
# [('of', ['henry_V', 'king_lear', 'lady_susan', 'macbeth', 'merchant_of_venice', 'midsummer', 'northanger_abbey', 
# DF:  [('of', 18), ('shakespeare', 15), ('henry', 9)]
# IDF:  [('of', 0.0), ('shakespeare', 0.1823215567939546), ('henry', 0.6931471805599453)]

[('iwi', ['merchant_of_venice', 'richard_III']), ('calme', ['othello']), ('circle', ['northanger_abbey', 'king_lear', 'senseandsensibility', 'persuasion', 'romeo_and_juliet', 'henry_V', 'prideandpredjudice', 'tempest', 'mansfield_park', 'emma'])]
DF:  [('iwi', 3), ('matrimonial', 11), ('circle', 6)]
IDF:  [('iwi', 1.791759469228055), ('calme', 1.2809338454620642), ('circle', 1.0986122886681098)]
